In [ ]:
from IPython.lib import kernel
kernel.get_connection_file()


In [ ]:
FRMA_PLATFORMS = ['GPL' + str(x) for x in [
    570, 571, 96,
    1261,  # organism Mus musculus
    6244,  # getting strange errors. data in matrix
    # 1355,  # NO FRMA, Rattus norvegicus,
    6246,  # no frma package (?). read.affybatch: The affy package can process data from the Gene ST 1.x series of arrays,
    # but you should consider using either the oligo or xps packages, which are specifically
    # designed for these arrays.
    # there is no package called ‘mogene10stv1frmavecs’

    90, # organism Saccharomyces cerevisiae
    3921,
    # 5175, # no frma package (?). read.affybatch: The affy package is not designed for this array type. 
    # Please use either the oligo or xps package.
    # 5188, # no cdf frma packages
    8321,  # organism Mus musculus
]]

AGLIENT_PLATFORMS = ['GPL4133', 'GPL6480', 'GPL15849', 'GPL1708', 'GPL887', 'GPL4091', 'GPL9128', 'GPL7264', 'GPL11387',
             'GPL8687', 'GPL6848', 'GPL2879', 'GPL5477', 'GPL8841', 'GPL10123', 'GPL4093', 'GPL11386', 'GPL10806',
             'GPL8269', 'GPL10150', 'GPL8583', 'GPL15931', 'GPL4126', 'GPL10152', 'GPL16050', 'GPL2567', 'GPL9053',
             'GPL14550', 'GPL5325', 'GPL10808', 'GPL13691', 'GPL9075', 'GPL8736', 'GPL885', 'GPL9777', 'GPL7504',
             'GPL8693', 'GPL2873', 'GPL17077', 'GPL10734', 'GPL13953', 'GPL13607', 'GPL13685', 'GPL7015', 'GPL15560',
             'GPL18623', 'GPL10481', 'GPL16280', 'GPL8737', 'GPL11068']

ILLUMINA_PLATFORMS = list(map('GPL{}'.format, [
   10558, 6947
]))

In [ ]:
import sys

In [ ]:
sys.path.append('/home/ubuntu/lib')

In [ ]:
sys.path.append('/home/ubuntu/preproc-ng')

In [ ]:
!ls /home/ubuntu/lib

In [ ]:
sys.path

In [ ]:
from importlib import reload

In [ ]:
import elasticsearch

In [ ]:
# reload(metadb)
# reload(metadb.api)
# reload(metadb.api.elastic)

import metadb
import metadb.api
import metadb.api.elastic
from metadb.api import ElasticApi
# reload(metadb.api.elastic.ElasticApi)

In [ ]:
api = ElasticApi(elasticsearch.Elasticsearch(), 
                samples_index='samples_dev',
                series_index='series_dev')

In [ ]:
it = api.search_series(platform=FRMA_PLATFORMS, preprocessed_exists=0, data_source='geo', organism='Homo sapiens')

In [ ]:
type(it)

In [ ]:
len(it)

In [ ]:
%store it

In [ ]:
it4080 = it[40:80]

In [ ]:
it = None
del it

In [ ]:
import gc
gc.collect()

In [ ]:
def create_tasks_from_series(series):
    if not series['supplementary_files']:
        return []

    it = (dict(samples=[s['accession'] for s in series['samples'] if s['platform'] == platform],
                 accession=series['accession'],
                 platform=platform,
                 supplementary_files=[f['name'] for f in series['supplementary_files']]
                )
            for platform in series['platforms'])
    return [d for d in it if len(d['samples'])]


In [ ]:
import service
import messages
from preproc_manager import config


In [ ]:
manager = service.ServiceProxy(
    push_port=config.manager_pull_port,
    subscribe_port=config.manager_pub_port
)

In [ ]:
for series in it[40:80]:
    for task in create_tasks_from_series(series):
            manager.send(messages.PreprocDatasetMessage(id='', **task))